In [1]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(os.listdir("/content/drive/MyDrive/Final Year Project - Lung Cancer Prediction/Datasets/CT Scan Images/"))

['test', 'train']


In [ ]:
data_dir = Path("/content/drive/MyDrive/Final Year Project - Lung Cancer Prediction/Datasets/CT Scan Images/")
train_dir = data_dir / 'train'
test_dir = data_dir / 'test'

In [ ]:
def load_train():
    cancer_dir = train_dir / 'Cancerous'
    non_cancer_dir = train_dir / 'Non_Cancerous'

    cancer_cases = cancer_dir.glob('*.jpg')
    non_cancer_cases = non_cancer_dir.glob('*.jpg')

    train_data = []
    train_label = []

    for img in cancer_cases:
        train_data.append(img)
        train_label.append('CANCER')

    for img in non_cancer_cases:
        train_data.append(img)
        train_label.append('NON_CANCER')

    df = pd.DataFrame(train_data)
    df.columns = ['images']
    df['labels'] = train_label
    df = df.sample(frac=1).reset_index(drop=True)
    return df

In [ ]:
train_data = load_train()

In [ ]:
def load_test():
  cancer_dir = test_dir/'Cancerous'
  non_cancer_dir = test_dir/'Non_Cancerous'

  cancer_cases = cancer_dir.glob('*.jpg')
  non_cancer_cases = non_cancer_dir.glob('*.jpg')

  test_data = []
  test_label = []

  for img in cancer_cases:
    test_data.append (img)
    test_label.append('CANCER')

  for img in non_cancer_cases:
    test_data.append (img)
    test_label.append('NON_CANCER')

  df = pd.DataFrame(test_data)
  df.columns = ['images']
  df['labels'] = test_label
  df = df.sample(frac=1).reset_index(drop=True)
  return df

In [ ]:
test_data = load_test()

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))

In [ ]:
model.add(Dense(2, activation='softmax'))

In [ ]:
def data_gen(data, batch_size):
    while True:
        batch_indices = np.random.choice(len(data), batch_size)
        batch_data = data.iloc[batch_indices]

        images = []
        labels = []
        for index, row in batch_data.iterrows():
            image_path = row['images']
            label = row['labels']

            img = cv2.imread(str(image_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
            img = img.astype('float32') / 255.0  # Normalize and convert to float32

            images.append(img)
            labels.append(label)

        images = np.array(images)
        labels_one_hot = to_categorical(pd.Categorical(labels).codes, num_classes=2)

        yield images, labels_one_hot


In [ ]:
batch_size = 16
nb_epochs = 5
train_gen = data_gen(data=train_data, batch_size=batch_size)
nb_train_steps = train_data.shape[0] // batch_size

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_gen,
    steps_per_epoch=nb_train_steps,
    epochs=nb_epochs
)

Epoch 1/5


ValueError: ignored